In [2]:
import planet
import asyncio
import os
import getpass
from planet import Auth

In [7]:
user = input("Username: ")
pw = getpass.getpass()
auth = Auth.from_login(user,pw)
auth.store()

Interaction with the server is done through sessions that automatically implement rate limits.
Do not use multiple Sessions as it can cause collisions and bypass rate-limiting.

In [1]:
async def create_and_download(client, order_detail, directory):
   with planet.reporting.StateBar(state='creating') as reporter:
       order = await client.create_order(order_detail)
       reporter.update(state='created', order_id=order['id'])
       await client.wait(order['id'], callback=reporter.update_state)

   await client.download_order(order['id'], directory, progress_bar=True)

async def main():
   async with planet.Session() as sess:
       cl = sess.client('orders')

       # Create the order request
       request = create_request()

       # Create and download the order
       order = await create_and_download(cl, request, DOWNLOAD_DIR)

Creating Orders:

In [ ]:
def create_request():
    # This is your area of interest for the Orders clipping tool
    oregon_aoi = {
       "type":
       "Polygon",
       "coordinates": [[[-117.558734, 45.229745], [-117.452447, 45.229745],
                        [-117.452447, 45.301865], [-117.558734, 45.301865],
                        [-117.558734, 45.229745]]]
   }

   # In practice, you will use a Data API search to find items, but
   # for this example take them as given.
   oregon_items = ['20200909_182525_1014', '20200909_182524_1014']

   oregon_order = planet.order_request.build_request(
       name='oregon_order',
       products=[
           planet.order_request.product(item_ids=oregon_items,
                                        product_bundle='analytic_udm2',
                                        item_type='PSScene')
       ],
       tools=[planet.order_request.clip_tool(aoi=oregon_aoi)])

   return oregon_order